In [1]:
import pandas as pd
import numpy as np
import time
import re

pd.options.display.max_columns = 100

# 合并add

In [2]:
add1 = pd.read_pickle(r'D:\zhihu data\zhihu II\comment\comments_df_added.pkl')
add2 = pd.read_pickle(r'D:\zhihu data\zhihu II\comment\comments_df_added_added.pkl')
add3 = pd.read_pickle(r'D:\zhihu data\zhihu II\comment\comments_df_added_t3.pkl')

In [3]:
add = add1.append([add2,add3], ignore_index = True)

In [4]:
del [add1,add2,add3]

In [5]:
add.shape  # (778947, 31)

(778947, 31)

In [6]:
add.drop('disliked', axis = 1, inplace=True)

In [7]:
add.shape  # (778947, 30)

(778947, 30)

In [8]:
#add.isnull().sum()  # 无空值

In [9]:
add.dtypes

answerid                object
allow_delete            object
allow_like              object
allow_reply             object
allow_vote              object
author_badge            object
author_gender           object
author_headline         object
author_id               object
author_is_advertiser    object
author_is_org           object
author_name             object
author_type             object
author_url_token        object
author_user_type        object
author_role             object
can_collapse            object
can_recommend           object
collapsed               object
content                 object
created_time            object
featured                object
commentid               object
is_author               object
is_delete               object
is_parent_author        object
resource_type           object
reviewing               object
vote_count              object
voting                  object
dtype: object

In [ ]:
add.head()

# 改变add数据类型

In [10]:
def Trans(dataframe, glist):
    for i in glist:
        dataframe[i].replace('True',1,inplace=True)
        dataframe[i].replace('False',0,inplace=True)
        dataframe[i].replace(True,1,inplace=True)
        dataframe[i].replace(False,0,inplace=True)

In [11]:
glist = ['allow_delete','allow_like','allow_vote','allow_reply',
         'author_is_advertiser','author_is_org','can_collapse',
         'can_recommend','collapsed','featured','is_author',
         'is_delete','is_parent_author','reviewing','voting']

In [12]:
Trans(add, glist)

In [13]:
add.dtypes

answerid                object
allow_delete              bool
allow_like                bool
allow_reply               bool
allow_vote                bool
author_badge            object
author_gender           object
author_headline         object
author_id               object
author_is_advertiser      bool
author_is_org             bool
author_name             object
author_type             object
author_url_token        object
author_user_type        object
author_role             object
can_collapse              bool
can_recommend             bool
collapsed                 bool
content                 object
created_time            object
featured                  bool
commentid               object
is_author                 bool
is_delete                 bool
is_parent_author          bool
resource_type           object
reviewing                 bool
vote_count              object
voting                    bool
dtype: object

In [ ]:
add.head()

In [14]:
add['author_gender'] = add.author_gender.astype('int64')
add['answerid'] = add.answerid.astype('int64')
add['commentid'] = add.commentid.astype('int64')
add['vote_count'] = add.vote_count.astype('int64')

In [15]:
add.dtypes

answerid                 int64
allow_delete              bool
allow_like                bool
allow_reply               bool
allow_vote                bool
author_badge            object
author_gender            int64
author_headline         object
author_id               object
author_is_advertiser      bool
author_is_org             bool
author_name             object
author_type             object
author_url_token        object
author_user_type        object
author_role             object
can_collapse              bool
can_recommend             bool
collapsed                 bool
content                 object
created_time            object
featured                  bool
commentid                int64
is_author                 bool
is_delete                 bool
is_parent_author          bool
resource_type           object
reviewing                 bool
vote_count               int64
voting                    bool
dtype: object

# 去掉add里乱码，和commentid重复项

In [16]:
mess = add['content'].str.contains('[çĽäşĽŻşĺäşŹčéĄśäčŻşăďźçšĺçłäşŻćŠťĽĺçéźćąĺłčżâŚâäťĽčäčľćçźéŕĽŕĽĺ]', regex=True)

In [17]:
mess_index = add.index.isin(add.loc[mess].index)

In [18]:
add = add.loc[~mess_index]

In [19]:
add.shape  # 删掉778947 - 599412 = 179535个乱码值

(599412, 30)

In [ ]:
add.loc[add.commentid.duplicated(keep=False), :].sort_values('commentid')

In [20]:
add.drop_duplicates(subset = ['commentid'], inplace = True)

In [21]:
add.shape # 删掉599412 - 589347 = 10065个commentid重复值

(589347, 30)

# 删掉在comment.csv大表里有的commentid


## 读大表：删多余列，去content为空，换数据类型，去commentid重复行

In [22]:
df = pd.read_csv(r'D:\zhihu data\zhihu II\comment\comments.csv',encoding = 'utf-8')

C:\Users\Xiaoning_mkt\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,7,10,11,17,18,19,23,24,25,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [24]:
df.shape

(18756149, 30)

In [25]:
df.dropna(subset=['content','answerid'], how = 'any', inplace=True)

In [26]:
df.shape # 删掉了 18756149 - 18748721 = 7428

(18748721, 30)

In [27]:
df.index = range(len(df))

In [28]:
Trans(df, glist)  # 函数见上，即换成Bool类型

# 只有 0 1 的自动变bool
# 只有0，只有1的自动变int 
# 带有空值的为 float

In [29]:
df.dtypes

allow_delete              int64
allow_like                 bool
allow_reply                bool
allow_vote                 bool
answerid                 object
author_badge             object
author_gender            object
author_headline          object
author_id                object
author_is_advertiser       bool
author_is_org              bool
author_name              object
author_role              object
author_type              object
author_url_token         object
author_user_type         object
can_collapse               bool
can_recommend              bool
collapsed                  bool
commentid               float64
content                  object
created_time            float64
featured                float64
is_author               float64
is_delete               float64
is_parent_author        float64
resource_type            object
reviewing               float64
vote_count              float64
voting                  float64
dtype: object

In [30]:
df['author_gender'] = df.author_gender.astype('int64')
df['answerid'] = df.answerid.astype('int64')
df['commentid'] = df.commentid.astype('int64')

In [ ]:
df.loc[df.commentid.duplicated(keep=False), :].sort_values('commentid')

In [31]:
df.drop_duplicates(subset = ['commentid'], inplace = True)

In [32]:
 # 去掉 18748721 -18158812 = 589909 个重复行

(18158812, 30)

## 筛选出大表里已有的commentid，缩小add   
原来：589347

In [ ]:
df.head()

In [33]:
add = add[~add.commentid.isin(list(df['commentid']))]

In [34]:
add.shape  # 减去485128个数据

(104219, 30)

# 合并原csv和add，记为df

(18158812, 30)
(104219, 30)
合并为：
(18263031, 30)

In [35]:
df = df.append(add, ignore_index = True)

In [238]:
df.shape

(18263031, 30)

In [ ]:
# df.commentid.duplicated().sum()

In [ ]:
# df.isnull().sum()

In [ ]:
# df.loc[df.author_name.isnull()]

# 这里少做了一步，没把answerid放进来呀！！！

# 分词 : 均用test = df[888456:888654] 做测试


## 先去掉Html代码

In [43]:
df["content"]= df["content"].map(lambda x: re.sub(r'</?\w+[^>]*>','',x) if type(x) == unicode else  x)

## jieba分词

返回空格连接的字符串形式

In [45]:
import jieba

In [231]:
def split_content(row):
    try:
        l = jieba.lcut(row)
        t = ' '.join(l)      
        return t
    except:
        return ''

In [239]:
#test = df[888456:888654]

In [240]:
#test

,allow_delete,allow_like,allow_reply,allow_vote,answerid,author_badge,author_gender,author_headline,author_id,author_is_advertiser,author_is_org,author_name,author_role,author_type,author_url_token,author_user_type,can_collapse,can_recommend,collapsed,commentid,content,created_time,featured,is_author,is_delete,is_parent_author,resource_type,reviewing,vote_count,voting
888456,0,True,True,True,23885283,[],1,写报告小王子,96ef309053de2853adc988f7ac966460,False,False,王星,normal,people,wang-xing-70-69,people,False,False,False,54011949,估计要答案被和谐……答主保重，多吃茶叶蛋,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,0.0,0.0
888457,0,True,True,True,17181460,[],1,iOS工程师,86cfc5976b379acb036dafbafc621397,False,False,肖玉龙,author,people,xiao-yu-long,people,False,False,False,54011950,山针,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,1.0,0.0
888458,0,True,True,True,22182699,[],1,Enrich myself,d6703863c0b25ad409ad8c144e89b7df,False,False,系安,normal,people,xi-an-70,people,False,False,False,54011957,写的是圣教序。临的不错。看得出有功底。,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,0.0,0.0
888459,0,True,True,True,23908670,[],1,爱求知，喜见闻，恶争辩，好躬行。,12e6484251835d5ff8c1a713d9b02932,False,False,子午,author,people,zi-wu-53,people,False,False,False,54011966,先培养兴趣吧，中医是一个知识体系，简而言之就是理法方药，当你看到或者体验到真的疗效才会坚定兴...,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,1.0,0.0
888460,0,True,True,True,17581657,[],0,web desgin,3838fd6e4c7c35560c730c1f9d0337fe,False,False,Susan,normal,people,shu-xing-49,people,False,False,False,54012005,我心中的教育，是帮助一个个体拥有完善的自我、人格和追求。是培养一个能够独立、有理想、对社会有...,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,0.0,0.0
888461,0,True,True,True,23910079,[],1,Quora翻译微信公众号：transq,e3e27613a4b59f0304df9ce6c5c5b016,False,False,张志云,author,people,zhang-zhi-yun-007,people,False,False,False,54012008,对了，这个方法，我已经实践过了。很容易实践，而且效果很不错。,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,0.0,0.0
888462,0,True,True,True,22720947,[],1,酒店,89e781e44cc9450dbd0f365747116467,False,False,袁松柏,normal,people,yuan-song-bo,people,False,False,False,54012014,擦……,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,0.0,0.0
888463,0,True,True,True,23899980,[],1,Smile & Reborn...,8810be702dbf812a0016ab52b2131c57,False,False,迦箬,normal,people,alien,people,False,False,False,54012015,银之匙……,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,4.0,0.0
888464,0,True,True,True,20779585,[],1,三十而立,c9ae1ee39b8871dce352a6daedee2ce2,False,False,李继刚,normal,people,lijigang.com,people,False,False,False,54012035,好厉害的书架，不过好像书桌有点不配套啊。,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,0.0,0.0
888465,0,True,True,True,23897907,[],0,在你的故事里,3675ec92439466f1d3e9116a9343762f,False,False,徐心雨,author,people,xu-xin-yu-83,people,False,False,False,54012052,忠犬八公也是妥妥的,1.39609e+09,0.0,0.0,0.0,0.0,answer,0.0,0.0,0.0


In [241]:
#%%time
#test['content_split1'] = test['content'].apply(split_content)

Wall time: 105 ms


C:\Users\Xiaoning_mkt\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [242]:
#test['content_split1']

888456                     估计 要 答案 被 和谐 … … 答主 保重 ， 多 吃 茶叶蛋
888457                                                   山针
888458                      写 的 是 圣教序 。 临 的 不错 。 看得出 有 功底 。
888459    先 培养 兴趣 吧 ， 中医 是 一个 知识 体系 ， 简而言之 就是 理法 方药 ， 当 ...
888460    我 心中 的 教育 ， 是 帮助 一个 个体 拥有 完善 的 自我 、 人格 和 追求 。 ...
888461    对 了 ， 这个 方法 ， 我 已经 实践 过 了 。 很 容易 实践 ， 而且 效果 很 ...
888462                                                擦 … …
888463                                              银之匙 … …
888464                     好 厉害 的 书架 ， 不过 好像 书桌 有点 不 配套 啊 。
888465                                        忠犬八公 也 是 妥妥 的
888466                                             总结 的 很 好
888467                       没 觉得 揉 过 以后 眼睛 很红 很 不 舒服 ？ ？ ？
888468                                 是 的 ， 没错 。 万物 皆 平等 。
888469                        某人 在 革命 时期 牺牲 ， 遗孤 被 周总理 抚养 。
888470    工行 的 系统 现在 会 自动 联想 后续 代码 了 ？ 比 我 在 的 时候 高大 上 多拉 。
888471          逗比 ， retina 意思 就是 ppi 超过 300 ， s3 就是 retina
888472    中医 的 优势 就是 不能 做 标准化 治疗 方案 ， 这 

In [ ]:
%%time
df['content_split'] = df['content'].apply(split_content)

In [ ]:
df.head()

## 去掉stopwords的jieba分词

返回空格连接的字符串形式

In [197]:
stop = pd.read_csv(r'C:\Users\Xiaoning_mkt\Desktop\stopword.csv', encoding = 'utf-8', header= None)

In [207]:
stoplist = list(stop[0])

In [234]:
def split_content2(row):
    split_str = ''
    try:
        for word in jieba.cut(row):
            if word not in stoplist:
                split_str += word + ' '
            else:
                pass
        return split_str.encode('utf-8')
    except:
        return ''

In [ ]:
%%time
df['content_split'] = df['content'].apply(split_content2)

In [243]:
#%%time
#test['content_split2'] = test['content'].apply(split_content2)

Wall time: 167 ms


C:\Users\Xiaoning_mkt\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [244]:
#test['content_split2']

888456                                估计 答案 和谐 答主 保重 吃 茶叶蛋 
888457                                                  山针 
888458                                     写 圣教序 不错 看得出 功底 
888459    先 培养 兴趣 中医 一个 知识 体系 理法 方药 看到 体验 真的 疗效 坚定 兴趣 学习...
888460    心中 教育 帮助 一个 个体 拥有 完善 自我 人格 追求 培养 一个 能够 独立 理想 社...
888461                                方法 已经 实践 容易 实践 效果 不错 
888462                                                   擦 
888463                                                 银之匙 
888464                                 好 厉害 书架 好像 书桌 有点 配套 
888465                                             忠犬八公 妥妥 
888466                                                总结 好 
888467                                    觉得 揉 以后 眼睛 很红 舒服 
888468                                          没错 万物 皆 平等 
888469                               某人 革命 时期 牺牲 遗孤 周总理 抚养 
888470                          工行 系统 现在 自动 联想 后续 代码 高大 多拉 
888471                   逗比 retina 意思 ppi 超过 300 s3 retina 
888472    中医 优势 做 标准化 治疗 方案 考验 中医师 水平 病人

In [ ]:
to_excel()

In [ ]:
# 不要用 try except, 要看看报错是啥呐 ~ 